In [59]:
from IPython.display import HTML

HTML('''<script> 
code_show=true;
function code_toggle() { 
  if (code_show){
    $('div.input').hide();
    $('div.prompt').hide();
  } 
  else { 
    $('div.prompt').show();
    $('div.input').show();
    } code_show = !code_show ;
    
}

$( document ).ready(code_toggle);
</script>
<style>
input {
    width : 100%;
    background-color: white;
    border: none;
    color: white;
    padding: 1px 1px;
    margin: 1px 1px;
    cursor: pointer;
}
</style>

<form action="javascript:code_toggle()">
  <input type="submit" value="">
</form>''')

# 2017 Plan
- Reinforcement Learning 
 - 
- Paper Reading
- Personal AI project ( investment )
- Data Scientist Blog Refine 


In [60]:

html = """
<div id="main" style="width: 600px;height:400px;"></div>

<script type="text/javascript">   

//
require.config({
         paths:{
            echarts: ['//cdn.bootcss.com/echarts/3.4.0/echarts.min'],
         }
    });

//
require(['echarts'],function(ec){
  //
  var myChart = ec.init(document.getElementById('main'));

  //
  var option = {
            title: {
                text: 'Kent'
            },
            tooltip: {},
            legend: {
                data:['销量']
            },
            xAxis: {
                data: ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
            },
            yAxis: {},
            series: [{
                name: '销量',
                type: 'bar',
                data: [5, 20, 36, 10, 10, 20]
            }]
        };

  //  
  myChart.setOption(option);
         });
    </script>
"""

In [61]:
from IPython.display import HTML
HTML(html)